In [0]:
USE CATALOG workspace;
USE SCHEMA taxi_assignment_db;

SHOW TABLES;

database,tableName,isTemporary
taxi_assignment_db,taxi_bronze,false
taxi_assignment_db,taxi_gold_top_fares,false
taxi_assignment_db,taxi_silver_clean,false
taxi_assignment_db,taxi_silver_weekly_stats,false


Bronze layer ingestion check

In [0]:
SELECT count(*) as total_raw_records FROM taxi_bronze;

total_raw_records
60


Silver Layer Quality Expectation Check

In [0]:
SELECT * FROM taxi_silver_clean WHERE trip_distance <= 0;

ride_id,timestamp,passenger_id,trip_distance,fare_amount,_rescued_data,price_per_mile,is_suspicious,week_start


In [0]:
SELECT COUNT(*) FROM taxi_silver_clean;

--- 2 rows dropped because of constraints (60 -> 58)

COUNT(*)
58


Silver Layer Logic & Data Type Check

In [0]:
DESCRIBE taxi_silver_clean;

col_name,data_type,comment
ride_id,string,null
timestamp,string,null
passenger_id,string,null
trip_distance,double,null
fare_amount,double,null
_rescued_data,string,null
price_per_mile,double,null
is_suspicious,boolean,null
week_start,timestamp,null


In [0]:
SELECT ride_id, trip_distance, fare_amount, is_suspicious 
FROM taxi_silver_clean 
WHERE is_suspicious = true;

ride_id,trip_distance,fare_amount,is_suspicious
ride_1_2151,1.2,85.06,true


Silver Layer Weekly Aggregates

In [0]:
SELECT week_start, total_rides, round(total_revenue, 2) as revenue
FROM taxi_silver_weekly_stats
ORDER BY week_start DESC;

week_start,total_rides,revenue
2025-11-24T00:00:00.000Z,3,183.92
2025-11-17T00:00:00.000Z,55,2430.94


Gold Layer

In [0]:
SELECT * FROM taxi_gold_top_fares;

ride_id,timestamp,passenger_id,fare_amount,is_suspicious
ride_3_5690,2025-11-24 12:31:05.622426,74,96.58,false
ride_2_9237,2025-11-23 12:31:05.527407,18,95.08,false
ride_1_2887,2025-11-23 12:31:04.476569,22,89.92,false


In [0]:
-- Querying the internal DLT logs to show exactly how many bad records were dropped.
SELECT 
  timestamp,
  details:flow_progress.metrics.num_output_rows as rows_passed,
  details:flow_progress.data_quality.dropped_records as rows_dropped
FROM event_log(TABLE(taxi_silver_clean))
WHERE details:flow_progress.data_quality.dropped_records > 0
ORDER BY timestamp DESC
LIMIT 5;

timestamp,rows_passed,rows_dropped
2025-11-24T13:31:26.450Z,58,2
2025-11-24T13:13:35.757Z,56,4
